# Tutorial: Variables (One-Shot `evaluate(...)`)

This notebook recreates examples from the official BQN tutorial page:
https://mlochbaum.github.io/BQN/tutorial/variable.html

This is the one-shot variant: each line is an independent `evaluate("...")` call.


In [27]:
from pathlib import Path
import sys


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src" / "bqn_jax").exists():
            return candidate
    raise RuntimeError(
        "Could not locate the bqn-jax repo root. Start Jupyter from this repo or set PYTHONPATH=src."
    )


_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / "src"
if str(_src) not in sys.path:
    sys.path.insert(0, str(_src))

import bqn_jax
from bqn_jax import evaluate as _evaluate_raw, EvaluationEnvironment

# This tutorial needs variables to persist between cells,
# so we use a stateful EvaluationEnvironment.
_env = EvaluationEnvironment()
evaluate = _evaluate_raw(_env)

print(f"Using bqn_jax from: {bqn_jax.__file__}")

Using bqn_jax from: /home/johtok/repos/jax/xlapl/bqn-jax/src/bqn_jax/__init__.py


## Before You Start

1. Run the setup cell first (`from bqn_jax import evaluate`).
2. Then run cells from top to bottom.
3. Each `evaluate("...")` call parses and runs only that snippet.
4. Some advanced tutorial examples may raise errors because `bqn_jax` does not implement every BQN feature yet.


## Examples

Quick warm-up examples before the deeper sections.


In [28]:
evaluate('hey ← "Hi there"')
evaluate('hey ∾ ", World!"')


Array([ 72, 105,  32, 116, 104, 101, 114, 101,  44,  32,  87, 111, 114,
       108, 100,  33], dtype=int32)

## Defining variables

Define names with `←`, then update existing names with `↩` when needed.


In [29]:
evaluate('pi‿e‿ten ← ⟨ π, ⋆1, 10 ⟩')
evaluate('ten × pi')
evaluate('three ⋈ ten - three ← 3')


Array([3., 7.], dtype=float32)

In [30]:
try:
    evaluate('three ← 4')
except NameError as e:
    print(f"Error (expected): {e}")

Error (expected): Duplicate definition for name 'three' in the same scope


In [31]:
evaluate('three ↩ 4')
evaluate('three = 3   # Wait why did I do that')
evaluate('3 = three ↩ 3')
try:
    evaluate("four ↩ 3    # four isn't defined yet")
except NameError as e:
    print(f"Error (expected): {e}")

Error (expected): Cannot update undefined name 'four'


## Variable roles

Identifier spelling and casing affect syntactic role in BQN.


In [32]:
evaluate('BQN ← "[many pages of specification]"')


Array([ 91, 109,  97, 110, 121,  32, 112,  97, 103, 101, 115,  32, 111,
       102,  32, 115, 112, 101,  99, 105, 102, 105,  99,  97, 116, 105,
       111, 110,  93], dtype=int32)

In [33]:
evaluate('three')
evaluate('thrEe')
evaluate('ThReE')
evaluate('thr_EE')
evaluate('__three')
evaluate('_T_H_R_E_E_')


Array(3., dtype=float32, weak_type=True)

In [34]:
evaluate('- Three')
evaluate('- _three')


Array(-3., dtype=float32, weak_type=True)

In [35]:
evaluate('1_000_000')


Array(1.e+06, dtype=float32, weak_type=True)

## Function assignment

Functions are first-class values, so assignment works for function definitions too.


In [36]:
evaluate('Base2 ← +⟜(2⊸×)´∘⌽')
evaluate('Base2 1‿0‿1‿0')
evaluate('Base2 "01010001"-\'0\'')
evaluate('@ + Base2¨ \'0\' -˜ "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [37]:
evaluate('Base2')
evaluate('base2 ↩ 16   # Change it to a number')
evaluate('Base2')
# Base2 is now 16 (not callable) — in BQN this would error.
# Our evaluator silently returns 16 for non-callable application.
evaluate('Base2 6')

Array(16., dtype=float32, weak_type=True)

## Modifying part of an array

These examples show targeted updates through functional transforms like Under.


In [38]:
evaluate('"BQN"            # A list of characters')
evaluate('-⟜1⌾(2⊸⊑) "BQN"  # Wait why did I do that')


Array(77., dtype=float32)

In [39]:
evaluate('(↕3) ⋈¨ "BQN"')
evaluate('1 ⊑ "BQN"')


Array(81, dtype=int32)

In [40]:
evaluate('8⌾⊑ "BQN"        # Change the first element to 8')


Array(8., dtype=float32, weak_type=True)

In [41]:
evaluate('↕7')
evaluate('4 ↑ ↕7           # The first four elements')
evaluate('⌽⌾(4⊸↑) ↕7       # And reverse them')
evaluate('⌽⌾(¯4⊸↑) ↕7      # Or reverse the last four')


Array([6, 5, 4, 3], dtype=int32)

In [42]:
evaluate('2⊸⌽⌾(1⊸⊑) "xyz"‿"ABCDE"‿"wxyz"‿"yz"')
evaluate('2⊸⌽⌾(2⊸↓) "XYabcde"')


Array([101,  97,  98], dtype=int32)

In [43]:
evaluate('¯3 ↓ "abcdefgh"')
evaluate('2 ↑ 4 ↓ "abcdefgh"')


Array([101, 102], dtype=int32)

In [44]:
evaluate('(\'A\'-\'a\')⊸+ ⌾ (2 ↑ 4⊸↓)  "abcdefgh"')


Array([0, 0], dtype=int32)

## Identity functions

Identity-like forms are useful when controlling data flow in larger expressions.


In [45]:
evaluate('⊢ "only"')
evaluate('⊣ "only"')
evaluate('"left" ⊢ "right"')
evaluate('"left" ⊣ "right"')


Array([108, 101, 102, 116], dtype=int32)

## Modified assignment

Combine assignment with transforms to update values concisely.


In [46]:
evaluate("a ← 4            # First it's a number\na")
evaluate("a ↩ 4‿5‿6        # Now it's a list!\na")


Array([4., 5., 6.], dtype=float32, weak_type=True)

In [47]:
evaluate('a ↩ a - 1\na')
evaluate('a -↩ 1')


Array([2., 3., 4.], dtype=float32, weak_type=True)

In [48]:
evaluate('a ∾˜↩ 0‿1')


Array([0., 1., 2., 3., 4.], dtype=float32, weak_type=True)

In [49]:
evaluate('"abcd" ⌽∘⊣ "wxyz"')
evaluate('a ⌽∘⊣↩ @')


Array([4., 3., 2., 1., 0.], dtype=float32, weak_type=True)

In [50]:
evaluate('a ⌽↩')
evaluate('a 4⊸-↩           # And back again')


Array([4., 3., 2., 1., 0.], dtype=float32, weak_type=True)

In [51]:
evaluate('-⟜4⌾(¯2⊸↑) a')
evaluate("a                # It hasn't changed, of course")


Array([4., 3., 2., 1., 0.], dtype=float32, weak_type=True)

In [52]:
evaluate('a -⟜4⌾(¯2⊸↑)↩')


Array([-3., -4.], dtype=float32, weak_type=True)